In [4]:
import os
from dotenv import load_dotenv
import gradio as gr
from langchain_openai import ChatOpenAI
from langchain.chains import ConversationChain
from langchain.memory import ConversationEntityMemory
from langchain.memory.prompt import ENTITY_MEMORY_CONVERSATION_TEMPLATE

load_dotenv()

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
OPENAI_MODEL = "gpt-3.5-turbo"

llm = ChatOpenAI(openai_api_key=OPENAI_API_KEY, model_name=OPENAI_MODEL, temperature=0.0)
buffer = ConversationEntityMemory(llm=llm)
conversation = ConversationChain(llm=llm, memory=buffer, prompt=ENTITY_MEMORY_CONVERSATION_TEMPLATE)

# Define the function for generating itinerary based on user inputs
def generate_itinerary(City, Days, Activities_Per_Day, Top_Priority, Other_Preferences, Special_Requests):
    query = (
        f"""You are my personal travel agent. I'm planning a trip and would like you to prepare my
        itinerary for {Days} days in {City} with {Activities_Per_Day} activities per day.
        My top priority for this trip is {Top_Priority}, and I also like to do {', '.join(Other_Preferences)}
        with any special requests: {Special_Requests}. Please provide the itinerary by day in list format and include
        the addresses of each of the activities."""
    )
    itinerary = conversation.predict(input=query)
    return itinerary

def feedback_itinerary(user_feedback):
    query = f"I didn't like that itinerary because {user_feedback}. Please provide me a new itinerary based on the new feedback. Keep the original inputs."
    new_itinerary = conversation.predict(input=query)
    return new_itinerary

In [45]:
def update_visibility(radio_input):
    if radio_input == 'Yes':
        return gr.Textbox(visible=False), gr.Button(visible=False)
    else:
        return gr.Textbox(visible=True), gr.Button(visible=True)

theme = gr.themes.Soft()

with gr.Blocks(theme=theme) as app:
    with gr.Row():
        with gr.Column(scale=1, min_width=300):
            city = gr.Textbox(label="City")
            days = gr.Number(label="Days", step=1)
            activities_per_day = gr.Number(label="Activities Per Day", step=1)
        with gr.Column(scale=2, min_width=500):
            top_priority = gr.Radio(label="Top Priority", choices=["Music", "Food", "Museums", "Sports"])
            other_preferences = gr.CheckboxGroup(label="Other Preferences", choices=["Adventure", "Food", "Michelin Star Meals", "Museums", "Nature", "Sports", "Wine Tasting"])
            special_requests = gr.Textbox(label="Special Requests")
    with gr.Row():
        submit = gr.Button("Generate Itinerary")
        
    
    itinerary_output = gr.Textbox(label="Recommended Itinerary", interactive=False)

    with gr.Row():
        with gr.Column(scale=1, min_width=300):
            click_me = gr.Radio(['Yes','No'], label="Do you like this itinerary?")
        with gr.Column(scale=2, min_width=500):
            user_feedback = gr.Textbox(label="What didn't you like about it?", interactive=True)
    with gr.Row():
        submit2 = gr.Button("Generate New Itinerary")
   
    # Define callbacks
    submit.click(fn=generate_itinerary, inputs=[city, days, activities_per_day, top_priority, other_preferences, special_requests], outputs=itinerary_output)
    click_me.change(update_visibility, inputs=[click_me], outputs=[user_feedback, submit2])
    submit2.click(fn=feedback_itinerary, inputs=[user_feedback], outputs=itinerary_output)
    
app.launch()



Running on local URL:  http://127.0.0.1:7899

To create a public link, set `share=True` in `launch()`.
